<a href="https://colab.research.google.com/github/JenBanks8585/COVID-19/blob/master/Jen_Banks_Covid19_exploratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Imports

import pandas as pd
from datetime import datetime

In [0]:
#Reading csv files

confirmed=pd.read_csv('https://raw.githubusercontent.com/JenBanks8585/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
death=pd.read_csv('https://raw.githubusercontent.com/JenBanks8585/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
recovered=pd.read_csv('https://raw.githubusercontent.com/JenBanks8585/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [104]:
# Checking shapes, heads, etc

confirmed.shape, death.shape, recovered.shape

((463, 59), (463, 59), (463, 59))

In [76]:
confirmed.head(1)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20
0,NaN,Thailand,15.0,101.0,2,3,5,7,8,8,14,14,14,19,19,19,19,25,25,25,25,32,32,32,33,33,33,33,33,34,35,35,35,35,35,35,35,35,37,40,40,41,42,42,43,43,43,47,48,50,50,50,53,59,70,75,82,114,147


In [0]:
#confirmed.isnull().sum()

In [105]:
# Dropping some columns

confirmed_drop= confirmed.drop(['Province/State', 'Lat', 'Long'], axis = 1)
death_drop= death.drop(['Province/State', 'Lat', 'Long'], axis = 1)
recovered_drop= recovered.drop(['Province/State', 'Lat', 'Long'], axis = 1)
confirmed_drop.shape, death_drop.shape, recovered_drop.shape

((463, 56), (463, 56), (463, 56))

In [106]:
# Grouping by country

confirmed_drop_grouped=confirmed_drop.groupby('Country/Region').sum()
death_drop_grouped=death_drop.groupby('Country/Region').sum()
recovered_drop_grouped=recovered_drop.groupby('Country/Region').sum()
confirmed_drop_grouped.head(2)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21
Albania,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51


In [107]:
# Transposing dataframes, use date as index

confirmed_drop_grouped_trans= confirmed_drop_grouped.T
death_drop_grouped_trans= death_drop_grouped.T
recovered_drop_grouped_trans= recovered_drop_grouped.T

confirmed_drop_grouped_trans.shape


(55, 156)

In [0]:
# Converting date from str to datetime

confirmed_drop_grouped_trans.index= pd.to_datetime(confirmed_drop_grouped_trans.index)
death_drop_grouped_trans.index= pd.to_datetime(death_drop_grouped_trans.index)
recovered_drop_grouped_trans.index= pd.to_datetime(recovered_drop_grouped_trans.index)

In [81]:
confirmed_drop_grouped_trans.tail()

Country/Region,Afghanistan,Albania,Algeria,Andorra,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahrain,Bangladesh,Belarus,Belgium,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Brazil,Brunei,Bulgaria,Burkina Faso,Cambodia,Cameroon,Canada,Central African Republic,Chile,China,Colombia,Congo (Brazzaville),Congo (Kinshasa),Costa Rica,Cote d'Ivoire,Croatia,Cruise Ship,Cuba,Cyprus,Czechia,Denmark,...,Republic of the Congo,Reunion,Romania,Russia,Rwanda,Saint Lucia,Saint Vincent and the Grenadines,San Marino,Saudi Arabia,Senegal,Serbia,Seychelles,Singapore,Slovakia,Slovenia,Somalia,South Africa,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Taiwan*,Tanzania,Thailand,The Bahamas,Togo,Trinidad and Tobago,Tunisia,Turkey,US,Ukraine,United Arab Emirates,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,occupied Palestinian territory
2020-03-12,7,23,24,1,0,19,4,0,128,302,11,195,3,12,314,0,1,2,11,52,11,7,2,3,2,117,0,23,80932,9,0,1,22,1,19,696,3,6,94,617,...,0,1,49,28,0,0,0,69,45,4,19,0,178,16,89,0,17,2277,2,0,0,599,652,49,0,70,0,1,0,7,1,1663,1,85,459,0,0,0,39,0
2020-03-13,7,33,26,1,1,31,8,2,200,504,15,189,3,27,559,0,1,3,13,151,37,23,2,5,2,193,0,43,80945,13,0,2,23,1,32,696,4,14,141,804,...,0,5,89,45,0,0,0,80,86,10,35,0,200,32,141,0,24,5232,6,1,0,814,1139,50,0,75,0,1,0,16,5,2179,3,85,802,0,0,0,47,0
2020-03-14,11,38,37,1,1,34,18,2,250,655,15,210,3,27,689,0,1,10,18,151,40,41,2,7,2,198,0,61,80977,22,0,2,26,1,38,696,4,26,189,836,...,0,6,123,59,1,1,1,80,103,10,46,2,212,44,181,0,38,6391,10,1,1,961,1359,53,0,82,0,1,2,18,5,2727,3,85,1144,4,0,2,53,0
2020-03-15,16,42,48,1,1,45,26,2,297,860,23,214,5,27,886,0,1,10,24,162,50,51,3,7,2,252,1,74,81003,34,1,2,27,1,49,696,4,26,253,875,...,0,7,131,63,1,2,1,101,103,24,48,2,226,54,219,0,51,7798,18,1,1,1022,2200,59,0,114,0,1,2,18,6,3499,3,98,1145,4,1,10,56,0
2020-03-16,21,51,54,2,1,56,52,2,377,1018,15,214,8,36,1058,1,1,11,25,200,54,52,15,7,4,415,1,155,81033,54,1,2,35,1,57,696,4,33,298,932,...,1,9,158,90,5,2,1,109,118,24,55,3,243,63,253,1,62,9942,28,1,1,1103,2200,67,1,147,1,1,4,20,18,4632,7,98,1551,8,6,17,61,0


In [0]:
# index the last date:

last_row_confirmed = pd.DataFrame(confirmed_drop_grouped_trans.loc[datetime(2020, 3, 16)])
last_row_death = pd.DataFrame(death_drop_grouped_trans.loc[datetime(2020, 3, 16)])
last_row_recovered = pd.DataFrame(recovered_drop_grouped_trans.loc[datetime(2020, 3, 16)])


In [83]:
last_row_confirmed['2020-03-16'].sort_values(ascending= False).head(10)

Country/Region
China             81033
Italy             27980
Iran              14991
Spain              9942
Korea, South       8236
Germany            7272
France             6650
US                 4632
Switzerland        2200
United Kingdom     1551
Name: 2020-03-16 00:00:00, dtype: int64

In [84]:
last_row_death.head(10)

,2020-03-16
Country/Region,
Afghanistan,0
Albania,1
Algeria,4
Andorra,0
Antigua and Barbuda,0
Argentina,2
Armenia,0
Aruba,0
Australia,3


In [110]:
last_row_confirmed=last_row_confirmed.reset_index()
last_row_death=last_row_death.reset_index()
last_row_recovered=last_row_recovered.reset_index()
last_row_confirmed

,Country/Region,2020-03-16 00:00:00
0,Afghanistan,21
1,Albania,51
2,Algeria,54
3,Andorra,2
4,Antigua and Barbuda,1
...,...,...
151,Uruguay,8
152,Uzbekistan,6
153,Venezuela,17
154,Vietnam,61


In [0]:
last_row_confirmed.columns=['Country/Region', 'Confirmed']
last_row_death.columns=['Country/Region', 'Death']
last_row_recovered.columns=['Country/Region', 'Recovered']

In [118]:
# Merging last counts

frame=[last_row_confirmed, last_row_death, last_row_recovered]
last_row_grouped= pd.merge(last_row_confirmed, last_row_death, how='outer', on= 'Country/Region')
last_row_grouped= pd.merge(last_row_grouped, last_row_recovered, how='outer', on= 'Country/Region')
last_row_grouped

,Country/Region,Confirmed,Death,Recovered
0,Afghanistan,21,0,1
1,Albania,51,1,0
2,Algeria,54,4,12
3,Andorra,2,0,1
4,Antigua and Barbuda,1,0,0
...,...,...,...,...
151,Uruguay,8,0,0
152,Uzbekistan,6,0,0
153,Venezuela,17,0,0
154,Vietnam,61,0,16


In [123]:
# Reshaping local_df to a tidy data

last_row_grouped_tidy = pd.melt(last_row_grouped,
                       ["Country/Region"],
                       var_name="Case Type",
                       value_name="Freq")
last_row_grouped_tidy = last_row_grouped_tidy.sort_values(by=["Country/Region"])
last_row_grouped_tidy.head(10)

,Country/Region,Case Type,Freq
0,Afghanistan,Confirmed,21
312,Afghanistan,Recovered,1
156,Afghanistan,Death,0
1,Albania,Confirmed,51
313,Albania,Recovered,0
157,Albania,Death,1
2,Algeria,Confirmed,54
314,Algeria,Recovered,12
158,Algeria,Death,4
3,Andorra,Confirmed,2


In [129]:
import altair as alt

base = alt.Chart(last_row_grouped_tidy).properties(width=1500)

line = base.mark_square().encode(
    x='Country/Region',
    y='Freq',
    color='Case Type'
)



line 

alt.Chart(...)